In [4]:
import numpy as np
import pandas as pd
import lightgbm as lgb, warnings
from sklearn.metrics import roc_auc_score 
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')

In [5]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [6]:

features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
param = {'bagging_freq': 5,
         'bagging_fraction': 0.331,
         'boost_from_average':'false',
         'boost': 'gbdt',
         'feature_fraction': 0.0405,
         'learning_rate': 0.0083,
         'max_depth': -1,
         'metric':'auc',
         'min_data_in_leaf': 80,
         'min_sum_hessian_in_leaf': 10.0,
         'num_leaves': 13,
         'num_threads': 8,
         'tree_learner': 'serial',
         'objective': 'binary',
         'verbosity': 1}
folds = StratifiedKFold(n_splits=15, shuffle=False, random_state=15)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
    


In [ ]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, train_df['target'].values)):
    print("Fold {}".format(fold_))
    trn_data, val_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=train_df['target'].iloc[trn_idx]), lgb.Dataset(train_df.iloc[val_idx][features], label=train_df['target'].iloc[val_idx])
    clf = lgb.train(param, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 4000)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(train_df['target'], oof)))


Fold 0
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.924417	valid_1's auc: 0.89887
[10000]	training's auc: 0.939862	valid_1's auc: 0.901961
[15000]	training's auc: 0.952448	valid_1's auc: 0.902155
Early stopping, best iteration is:
[11245]	training's auc: 0.943199	valid_1's auc: 0.902323
Fold 1
Training until validation scores don't improve for 4000 rounds.


In [ ]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values, 'target':predictions})
sub.to_csv("submission.csv", index=False)